### note: change all of the paths to your correct path when loading in data

In [67]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as scp
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
from sklearn.linear_model import LinearRegression
import os
import Levenshtein

In [68]:
pd.options.display.max_rows = 600
pd.options.display.max_columns = 85

In [69]:
home_tmp = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/mismatched_home.csv', header = None)
cdbg_tmp = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/mismatched_cdbg.csv', header = None)
mismatched_home = home_tmp.iloc[:,0]
mismatched_cdbg = cdbg_tmp.iloc[:,0]

In [70]:
mismatched_home.shape[0] + mismatched_cdbg.shape[0]

99

In [71]:
df_utilities2012 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2012.csv')
df_utilities2013 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2013.csv')
df_utilities2014 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2014.csv')
df_utilities2015 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2015.csv')
df_utilities2016 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2016.csv')
df_utilities2017 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2017.csv')
df_utilities2018 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2018.csv')
df_utilities2019 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2019.csv')
df_utilities_list = [df_utilities2012,df_utilities2013,df_utilities2014,df_utilities2015,
                    df_utilities2016,df_utilities2017,df_utilities2018,df_utilities2019]

In [72]:
list1 = []
list2 = []
for df_u in df_utilities_list:
    list1.append(df_u['Premise Address'])
for series_u in list1:
    for row in series_u:
        list2.append(row)
series_utilities = pd.Series(list2)

In [73]:
s_utilities = pd.Series(series_utilities.unique())

In [74]:
xlsx = pd.ExcelFile('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/All CED Funded Projects 2007-2017.xlsx')
df1_original = pd.read_excel(xlsx, 'HOME Funded Projects')
df2_original = pd.read_excel(xlsx, 'CDBG Funded Projects')
df_original = pd.concat([df1_original, df2_original])
df_original.index = range(850)

### helper functions

In [75]:
def lev_list(address_list, address, i):
    temp_list = []
    temp_list_u = []
    for u_address in address_list:
        if Levenshtein.distance(address, u_address.upper()) <= i:
            temp_list.append(address)
            temp_list_u.append(u_address.upper())
    temp_list_series = pd.Series(temp_list)
    temp_list_u_series = pd.Series(temp_list_u)
    return temp_list_u_series

In [76]:
def space_fix_utilities(s_util):
    s_util_to_series = []
    for address in s_util:
        full_address = address.split()
        length = len(full_address)
        word = full_address[0] + "  "
        for i in range(1, length-1):
            word += full_address[i] + " "
        word += " " + full_address[length-1]
        word = word.strip()
        s_util_to_series.append(word)
    return pd.Series(s_util_to_series)

In [77]:
def construct_fixed_addresses(list_series):
    tmp_to_series = []
    for (apt, dup, sno, snm, stp) in zip(list_series[0],list_series[1],list_series[2],
                                         list_series[3],list_series[4]):
        word = ""
        if sno != "NA":
            word += sno + " "
        if snm != "NA":
            word += snm + " "
        if stp != "NA":
            word += stp
        tmp_to_series.append(word.strip())
    return pd.Series(tmp_to_series)

In [78]:
def gen_list_none(address_list, mismatched, i):
    tmp_list = []
    for address in mismatched:
        if address == "UNKNOWN ADDRESS" or address == "SUPPRESSED ADDRESS":
            continue
        if lev_list(address_list, address, i).shape[0] == 0:
            tmp_list.append(address)
    return pd.Series(tmp_list)

### Read in more data

In [79]:
#s_utilities = space_fix_utilities(s_utilities)
df_parsehome = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/fixed_home_v02.csv')
df_parsecdbg = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/fixed_cdbg.csv')

In [80]:
s_parsehome = df_parsehome['Address']
s_parsecdbg = df_parsecdbg['Address']

In [81]:
def containsAPT(series):
    apt_list = []
    for address in series:
        if "APT" in address:
            apt_list.append(address)
    return pd.Series(apt_list)

In [82]:
def containsDUP(series):
    dup_list = []
    for address in series:
        full_address = address.split(' ')
        length = len(full_address)
        for i in range(length - 1):
            word = full_address[i]
            if len(word) == 1:
                if not ((word == 'N') or (word == 'E')
                or (word == 'S') or (word == 'W')):
                    dup_list.append(address)
    return pd.Series(dup_list)

In [83]:
def clean(series):
    tmp_list = []
    for address in series:
        full_address = address.split(' ')
        length = len(full_address)
        if any(char.isdigit() for char in full_address[length-1]):
            tmp_list.append(address)
        else:
            tmp_list.append("---")
    return pd.Series(tmp_list)

In [84]:
def splitter(series):
    apt_list = set(containsAPT(series))
    dup_list = set(containsDUP(series))
    sno_to_series = []
    snm_to_series = []
    stp_to_series = []
    apt_to_series = []
    dup_to_series = []

    for address in series:
        full_address = address.split()
        length = len(full_address)
        begin_index = 1
        end_index = length - 1
        #mutually exclusive apt and duplex
        if address == "UNKNOWN ADDRESS":
            sno_to_series.append("NA")
            snm_to_series.append("UNKNOWN ADDRESS")
            stp_to_series.append("NA")
            apt_to_series.append("NA")
            dup_to_series.append("NA")
            continue
        if address == "SUPPRESSED ADDRESS":
            sno_to_series.append("NA")
            snm_to_series.append("SUPPRESSED ADDRESS")
            stp_to_series.append("NA")
            apt_to_series.append("NA")
            dup_to_series.append("NA")
            continue
        if address in apt_list:
            apt_to_series.append(full_address[length-1])
            stp_to_series.append(full_address[length-3])
            end_index -= 2
        else:
            apt_to_series.append("NA")
            stp_to_series.append(full_address[length-1])
            
        if address in dup_list:
            dup_to_series.append(full_address[1])
            begin_index += 1
        else:
            dup_to_series.append("NA")
        sno_to_series.append(full_address[0])
        to_str = full_address[begin_index:end_index]
        word = ""
        for part in to_str:
            word += part + " "
        snm_to_series.append(word.strip())
    series_apt = pd.Series(apt_to_series)
    series_dup = pd.Series(dup_to_series)
    series_sno = pd.Series(sno_to_series)
    series_snm = pd.Series(snm_to_series)
    series_stp = pd.Series(stp_to_series)
    return (series_apt, series_dup, series_sno, series_snm, series_stp)

In [85]:
series_apt_home, series_dup_home, series_sno_home, series_snm_home, series_stp_home = splitter(s_parsehome)
series_apt_cdbg, series_dup_cdbg, series_sno_cdbg, series_snm_cdbg, series_stp_cdbg = splitter(s_parsecdbg)

In [86]:
home = [series_apt_home, series_dup_home, series_sno_home, series_snm_home, series_stp_home]
cdbg = [series_apt_cdbg, series_dup_cdbg, series_sno_cdbg, series_snm_cdbg, series_stp_cdbg]
for hom in home:
    print(hom.shape)
for cdb in cdbg:
    print(cdb.shape)

(286,)
(286,)
(286,)
(286,)
(286,)
(564,)
(564,)
(564,)
(564,)
(564,)


In [87]:
def checker(list_series):
    tmp_to_series = []
    for (apt, dup, sno, snm, stp) in zip(list_series[0],list_series[1],list_series[2],
                                         list_series[3],list_series[4]):
        word = ""
        if sno != "NA":
            word += sno + " "
        if dup != "NA":
            word += dup + " "
        word += snm + " "
        if stp != "NA":
            word += stp + " "
        if apt != "NA":
            word += "APT " + apt
        word = word.strip()
        tmp_to_series.append(word)
    return pd.Series(tmp_to_series)

In [88]:
s_parsehome[checker(home) != s_parsehome]

Series([], Name: Address, dtype: object)

In [89]:
df_all_addresses = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/all_addresses.csv')
df_all_addresses = df_all_addresses.drop('Unnamed: 0', axis = 1)
s_all_addresses = df_all_addresses['line1']

### Workflow

s_utilities is a list of all addresses that show up in the utilities data <br>
s_all_addresses is a list of all addresses in Albany <br>
df_original is a list of all housing projects <br>
lev_list takes in one of the two lists above, an address, and a threshold. <br>
If the address is within the threshold distance (string comparison) of an address in the two lists, it will print out.

In [90]:
mismatched_home

0             2620  S MADISON  ST
1               529  COCHRAN  AVE
2          2226  OLD DOMINION  DR
3             2701  DONCASTER  RD
4            250  BONNY VIEW  AVE
5            249  BONNY VIEW  AVE
6            248  BONNY VIEW  AVE
7            251  BONNY VIEW  AVE
8            253  BONNY VIEW  AVE
9            247  BONNY VIEW  AVE
10           254  BONNY VIEW  AVE
11           243  BONNY VIEW  AVE
12           245  BONNY VIEW  AVE
13           252  BONNY VIEW  AVE
14            2010  LAFAYETTE  CR
15              20  S JACKSON  ST
16               603  N HALEY  ST
17           1112  PEACHTREE  TER
18             2517  CARDINAL  DR
19            2006  LAFAYETTE  CR
20              818  WESTVIEW  DR
21              2501  REDWOOD  CR
22              933  RANDOLPH  DR
23            2201  CHAMPAGNE  DR
24              215  BROADWAY  ST
25              1003  HOLLAND  DR
26            2200  CHAMPAGNE  LN
27           1019  UNIVERSITY  DR
28            509  WESTOVER  BLVD
29            

In [91]:
mismatched_cdbg

0                         UNKNOWN ADDRESS
1                          400  EBONY  LN
2                    638  W HIGHLAND  AVE
3                         UNKNOWN ADDRESS
4                         UNKNOWN ADDRESS
5                         UNKNOWN ADDRESS
6                  230  S JACKSON ST  250
7                         UNKNOWN ADDRESS
8                         UNKNOWN ADDRESS
9                         UNKNOWN ADDRESS
10                       302  S CASON  ST
11                        UNKNOWN ADDRESS
12                     SUPPRESSED ADDRESS
13                     225  S JACKSON  ST
14                   1010  W LINCOLN  AVE
15    1401  E CAMPBELL ST/108 CARROLL  ST
16                     2572  CRESCENT  DR
17                     2572  CRESCENT  DR
18                       4835  IMPALA  LN
19                         507  EMILY  ST
20                    608  RESIDENCE  AVE
21                       614  ROADWAY  ST
22                       706  E TOWNE  RD
23                     SUPPRESSED 

### example (note the spacing difference between addresses in utilities and the full addr list)

In [100]:
lev_list(s_utilities, "1010 W LINCOLN AVE", 1)

0    1014 W LINCOLN AVE
1    1810 W LINCOLN AVE
2    1710 W LINCOLN AVE
3    1510 W LINCOLN AVE
4    1410 W LINCOLN AVE
5    1110 W LINCOLN AVE
6    1310 W LINCOLN AVE
7    1000 W LINCOLN AVE
8    1020 W LINCOLN AVE
dtype: object

In [98]:
lev_list(s_all_addresses, "1010 W LINCOLN AVE", 4)

0      1310 W LINCOLN AVE
1      1308 W LINCOLN AVE
2      1306 W LINCOLN AVE
3      1304 W LINCOLN AVE
4      1302 W LINCOLN AVE
5      1300 W LINCOLN AVE
6      1230 W LINCOLN AVE
7      1228 W LINCOLN AVE
8      1226 W LINCOLN AVE
9      1224 W LINCOLN AVE
10     1224 W LINCOLN AVE
11     1222 W LINCOLN AVE
12     1220 W LINCOLN AVE
13     1218 W LINCOLN AVE
14     1218 W LINCOLN AVE
15     1216 W LINCOLN AVE
16     1212 W LINCOLN AVE
17     1208 W LINCOLN AVE
18     1206 W LINCOLN AVE
19     1204 W LINCOLN AVE
20     1202 W LINCOLN AVE
21     1200 W LINCOLN AVE
22     1126 W LINCOLN AVE
23     1124 W LINCOLN AVE
24     1122 W LINCOLN AVE
25     1120 W LINCOLN AVE
26     1118 W LINCOLN AVE
27     1116 W LINCOLN AVE
28     1114 W LINCOLN AVE
29     1112 W LINCOLN AVE
30     1110 W LINCOLN AVE
31     1108 W LINCOLN AVE
32     1106 W LINCOLN AVE
33     1104 W LINCOLN AVE
34     1400 W LINCOLN AVE
35     1402 W LINCOLN AVE
36     1404 W LINCOLN AVE
37     1512 W LINCOLN AVE
38     1510 

### So in the above example, 2620 S Madison St is found in the list of all addresses, but not found in the utilities data. Record this fact in the google doc. Then, look up the project type in the housing project data to also record what type of project it was. I'm looking to see if there is any trend with demolition or disposition projects not being found in utilities data.

### example (note the capitalization difference)

In [55]:
df_original[df_original['Address'].str.contains('Residence')]

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
286,NaN,1008 East Residence,Albany,GA,NaN,2007,7,Emergency Repairs,578,Completed,CDBG,5671.14,5671.14,0.0
328,NaN,801 W Residence Ave,Albany,GA,NaN,2007,15,Public Service,572,Completed,CDBG,18572.87,18572.87,0.0
499,NaN,608 Residence Avenue,Albany,GA,NaN,2010,13,Demolition,941,Completed,CDBG,3341.25,3341.25,0.0
786,NaN,1505 E. Residence,Albany,GA,NaN,2016,5,Emergency Repairs,1470,Completed,CDBG,5884.47,5884.47,0.0
816,NaN,1329 E. Residence,Albany,GA,NaN,2017,20,Emergency Repairs,1518,Completed,CDBG,11896.43,11896.43,0.0


### Second to last merge

#### fix in ds

In [56]:
mismatched_cdbg

48                          400 EBONY LN
50                    638 W HIGHLAND AVE
93                  230 S JACKSON ST 250
156                     225 S JACKSON ST
159                   1010 W LINCOLN AVE
167    1401 E CAMPBELL ST/108 CARROLL ST
191                     2572 CRESCENT DR
192                     2572 CRESCENT DR
208                       4835 IMPALA LN
210                        507 EMILY AVE
213                    608 RESIDENCE AVE
214                       614 ROADWAY ST
216                        706 E TOWN RD
279                        743 E TOWN RD
318                732 W OGLETHORPE BLVD
334                  215 BROADWAY CT #13
335                  215 BROADWAY CT #10
368                   2116 W WADDELL AVE
380                       1306 NEWTON RD
410                242 E OGLETHORPE BLVD
412                244 E OGLETHORPE BLVD
414                246 E OGLETHORPE BLVD
416                248 E OGLETHORPE BLVD
418                250 E OGLETHORPE BLVD
420             

In [57]:
s_fixed_home = construct_fixed_addresses(home)
s_fixed_cdbg = construct_fixed_addresses(cdbg)
mismatched_home = s_fixed_home[~(s_fixed_home.isin(s_utilities.str.upper()))]
mismatched_cdbg = s_fixed_cdbg[~(s_fixed_cdbg.isin(s_utilities.str.upper()))]
length = len(mismatched_cdbg.index)
for i in mismatched_cdbg.index:
    if mismatched_cdbg[i] == "UNKNOWN ADDRESS" or mismatched_cdbg[i] == "SUPPRESSED ADDRESS":
        mismatched_cdbg.drop(i, inplace = True)

In [65]:
mismatched_home.shape

(21,)

In [66]:
mismatched_cdbg.shape

(44,)